In [1]:

import torch
import math
from utils.aihub_kpop_dataset import KpopImageDatasetwT
from torch.utils.data import DataLoader
from model import PoseHRNet
from torch import nn, Tensor
from torch.utils.tensorboard import SummaryWriter
import tqdm
from tqdm import tqdm_notebook

writer = SummaryWriter()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device ? : {}'.format(device))
mymodel = PoseHRNet('w32', num_joints=29)
mydataset = KpopImageDatasetwT()
mydataloader = DataLoader(dataset=mydataset, batch_size=128, shuffle=True)
myloss = nn.MSELoss()
myoptimizer = torch.optim.Adam(mymodel.parameters(), lr=1e-3)


/Users/junghyun/miniforge3/envs/torch_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device ? : cpu


In [3]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    
    for batch, (inputs, labels) in tqdm.tqdm(enumerate(mydataloader)):
        myoptimizer.zero_grad()
        
        pred = mymodel(inputs)
        
        loss = myloss(pred, labels)
        loss.backward()
        
        myoptimizer.step()
        
        running_loss += loss.item()
    
    return
        

In [4]:
EPOCHS = 5
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))
    mymodel.train(True)
    train_one_epoch(epoch)
    
torch.save(mymodel, 'latest_.pth')

EPOCH 1:


/var/folders/pn/7p2x9tx141qcnhcr_hphngt40000gn/T/ipykernel_95873/3857086708.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch, (inputs, labels) in tqdm_notebook(enumerate(mydataloader)):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html